## DSC550
##### Week3 assignment, Author: Xin Tang, 3/30/24

#### Part I. Using the TextBlob Sentiment Analyzer, as well as Vader

In [1]:
# Load packages first
import numpy as np 
import pandas as pd

In [2]:
# load the data frame
df = pd.read_csv('labeledTrainData.tsv', delimiter='\t')

In [3]:
df.head(5)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [14]:
# the IMDB rating < 5 results in a sentiment score of 0 (negative), and rating >=7 have a sentiment score of 1 (positive). 
# how many positive and negative review are there
Pos_count = df[df['sentiment']== 1]
Neg_count = df[df['sentiment']== 0]
print('count of positive review is:',len(Pos_count['sentiment']))
print('count of negative review is:',len(Neg_count['sentiment']))


count of positive review is: 12500
count of negative review is: 12500


In [7]:
#!pip install -U textblob


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\Daisy\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [6]:
#python -m textblob.download_corpora

SyntaxError: invalid syntax (2621292756.py, line 1)

In [4]:
#try to use textblob to classify the review
from textblob import TextBlob
review = df['review']
#review[1]
blob_review =[]
for item in df['review']:
    blob = TextBlob(item)
    blob_review.append(blob.sentiment.polarity)
#blob_review[:3]

In [5]:
pos_count = 0
neg_count = 0
for item in blob_review:
    if item >0:
        pos_count += 1
    else:
        neg_count +=1    

print('Positive review count is:', pos_count)
print('Negative review count is:',neg_count)

Positive review count is: 19000
Negative review count is: 6000


In [25]:
#create predicted sentinment
predict =[]
for item in blob_review:
    if item >0:
        predict.append(1)
    else:
        predict.append(0)
        
 
from sklearn.metrics import accuracy_score
print('accuracy of texblob module is:',accuracy_score(df['sentiment'], predict))

accuracy of texblob module is: 0.68528


###### Summary:
The original label show positive and negative review is half and half (12500: 12500). Using Textblob to predict the sentiment, the positive increased by 6500 and negative reduced by 6500. 
the accuracy is 0.68.
Per statistics, random guessing will yield half and half (accuracy is 0.5). so this method is better than guessing, but need true sentiment data to make a call.

#### Extra: try to assess sentiment using Vader

In [18]:
# import package
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [19]:
scores =[]
review2 = df['review']

In [20]:
for item2 in review2:
    result= analyser.polarity_scores(item2)
    scores.append(result['compound'])

In [21]:
len(scores)

25000

In [22]:
# Vader will tell how positive and negative a sentiment is, we can set up a neutral, but to compare with text blob,
# still keep 0 as criteria for positive and nagative
Vader_pos_count = 0
Vader_neg_count = 0
#Vader_neutral = 0
for item in scores:
    if item > 0:
        Vader_pos_count += 1
    elif item < 0 :
        Vader_neg_count += 1 
    else:    
        Vader_neutral += 1
print('Positive review count using Vader is:', Vader_pos_count)
print('Negative review count using Vader is:',Vader_neg_count)
#print('Neutral review count using Vader is:',Vader_neutral)

Positive review count using Vader is: 16591
Negative review count using Vader is: 8389


In [24]:
# calculate the accuracy
#create predicted sentinment
predict2 =[]
for score in scores:
    if score >0:
        predict2.append(1)
    else:
        predict2.append(0)
        
 
from sklearn.metrics import accuracy_score
print('accuracy of vader module is:', accuracy_score(df['sentiment'], predict2))

accuracy of vader module is: 0.6942


###### Summary:
Using Vader to predict the sentiment, there are 16591 positive and 8389 negative. 
accuracy is 0.6942, which is slightly better than textblob model. 

##### End of exercise part 1

### Part 2: Prepping Text for a Custom Model

In [3]:
# since we need to work on text, so clean up the text
# drop ID column
df.drop(columns = ['id'])
#check if any row is empty
print('NA check:',df.isnull().sum())
#check if any duplicate
print('duplicated check:',df.duplicated().sum())

NA check: id           0
sentiment    0
review       0
dtype: int64
duplicated check: 0


In [4]:
# clean up #1: remove html tags
import re

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
df['review'] = df['review'].apply(remove_html_tags)
df.head(5)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
# cleanup #2: remove all special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9\s]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text
df['review']=df['review'].apply(remove_special_characters)
df.head(5)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,The Classic War of the Worlds by Timothy Hines...
2,7759_3,0,The film starts with a manager Nicholas Bell g...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [6]:
# clean up #3: remove punctuation
#import unicodedata
#import sys
import string
c= df['review']
#punctuation = dict.fromkeys(
#    (i for i in range(sys.maxunicode)
#    if unicodedata.category(chr(i)).startswitch('P')
#    ),
#    None
#)
def remove_punc(text):
    test_str =text.translate(str.maketrans('', '', string.punctuation))
    return test_str
df['review']=df['review'].apply(remove_punc)
df.head(3)    

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,The Classic War of the Worlds by Timothy Hines...
2,7759_3,0,The film starts with a manager Nicholas Bell g...


In [7]:
# clean up #4: Convert all text to lowercase letters.
df['review'] = df['review'].apply(lambda x: x.lower())
df.head(4)

,id,sentiment,review
0,5814_8,1,with all this stuff going down at the moment w...
1,2381_9,1,the classic war of the worlds by timothy hines...
2,7759_3,0,the film starts with a manager nicholas bell g...
3,3630_4,0,it must be assumed that those who praised this...


In [8]:
# clean up #5: remove all stop words
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    processed_text = ' '.join(filtered_tokens)
    
    return processed_text
    
#try out new function
#r = df['review']
#r1 = r[1]
#r2 = remove_stopwords(r1)

df['review']=df['review'].apply(remove_stopwords)

In [10]:
# Apply NLTK’s PorterStemmer
#first tokenize the string into word
from nltk.stem.porter import  PorterStemmer

def tokenize(text):
    word = word_tokenize(text)
    return word

df['token'] =df['review'].apply(tokenize)

In [26]:
df['token']

0        [stuff, going, moment, mj, ive, started, liste...
1        [classic, war, worlds, timothy, hines, enterta...
2        [film, starts, manager, nicholas, bell, giving...
3        [must, assumed, praised, film, greatest, filme...
4        [superbly, trashy, wondrously, unpretentious, ...
                               ...                        
24995    [seems, like, consideration, gone, imdb, revie...
24996    [dont, believe, made, film, completely, unnece...
24997    [guy, loser, cant, get, girls, needs, build, p...
24998    [30, minute, documentary, buuel, made, early, ...
24999    [saw, movie, child, broke, heart, story, unfin...
Name: token, Length: 25000, dtype: object

In [11]:
# apply stemmer
#d =df['token']
#d[1]
porter = PorterStemmer()
def stem(text):
    stem = [porter.stem(word) for word in text]
    return stem

df['stem'] = df['token'].apply(stem)

In [27]:
df['stem']

0        [stuff, go, moment, mj, ive, start, listen, mu...
1        [classic, war, world, timothi, hine, entertain...
2        [film, start, manag, nichola, bell, give, welc...
3        [must, assum, prais, film, greatest, film, ope...
4        [superbl, trashi, wondrous, unpretenti, 80, ex...
                               ...                        
24995    [seem, like, consider, gone, imdb, review, fil...
24996    [dont, believ, made, film, complet, unnecessar...
24997    [guy, loser, cant, get, girl, need, build, pic...
24998    [30, minut, documentari, buuel, made, earli, 1...
24999    [saw, movi, child, broke, heart, stori, unfini...
Name: stem, Length: 25000, dtype: object

In [12]:
# create bag of words
#import library
from sklearn.feature_extraction.text import CountVectorizer


In [13]:
# the program will run into error if I transform df['stem'] directly, it works on df['review'] though. I need to loop it.
# system will give error" AttributeError: 'list' object has no attribute 'lower'
s=df['stem']
stem = []
for item in s:
 count = CountVectorizer()
 bag_of_words = count.fit_transform(item)
 stem.append(bag_of_words)   

In [16]:
print('There are %d rows in bag of words.' %(len(stem)))

There are 25000 rows in bag of words.


###### the number of rows this bag-of-words matrix is 25000, same as original data frame

In [18]:
# create tf-idf matrix on stemmed text
from sklearn.feature_extraction.text import TfidfVectorizer
e= df['stem']

tfidf_matrix = []
for item in e:
    tfidf = TfidfVectorizer()
    matrix = tfidf.fit_transform(item)
    tfidf_matrix.append(matrix)

In [52]:
len(tfidf_matrix)

25000

###### the dimension of tf-idf matrix is the same as the bag of words matrix

###### End of exercise part 2